In [151]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Cropping2D, Conv2D, MaxPooling2D, Dropout, Dense, Flatten,BatchNormalization
import os
import csv
import matplotlib.image as mpimg
import easydict
import cv2

### Data Exploration & Preprocessing

In [152]:
data_dir = './data'
data = pd.read_csv(os.path.join(data_dir,'driving_log.csv'))

In [153]:
data.head()

,/Users/kibaekjeong/Desktop/cloning data/IMG/center_2019_09_06_17_43_40_849.jpg,/Users/kibaekjeong/Desktop/cloning data/IMG/left_2019_09_06_17_43_40_849.jpg,/Users/kibaekjeong/Desktop/cloning data/IMG/right_2019_09_06_17_43_40_849.jpg,0,0.1,0.2,7.420227E-06
0,/Users/kibaekjeong/Desktop/cloning data/IMG/ce...,/Users/kibaekjeong/Desktop/cloning data/IMG/le...,/Users/kibaekjeong/Desktop/cloning data/IMG/ri...,-0.013971,0.034260,0.0,0.001163
1,/Users/kibaekjeong/Desktop/cloning data/IMG/ce...,/Users/kibaekjeong/Desktop/cloning data/IMG/le...,/Users/kibaekjeong/Desktop/cloning data/IMG/ri...,-0.028493,0.283845,0.0,0.139888
2,/Users/kibaekjeong/Desktop/cloning data/IMG/ce...,/Users/kibaekjeong/Desktop/cloning data/IMG/le...,/Users/kibaekjeong/Desktop/cloning data/IMG/ri...,-0.033318,0.481680,0.0,0.465651
3,/Users/kibaekjeong/Desktop/cloning data/IMG/ce...,/Users/kibaekjeong/Desktop/cloning data/IMG/le...,/Users/kibaekjeong/Desktop/cloning data/IMG/ri...,-0.034926,0.692577,0.0,1.022985
4,/Users/kibaekjeong/Desktop/cloning data/IMG/ce...,/Users/kibaekjeong/Desktop/cloning data/IMG/le...,/Users/kibaekjeong/Desktop/cloning data/IMG/ri...,-0.038695,0.881505,0.0,1.648629


In [154]:
row = ['center','left','right','steering','throttle','brake','speed']
with open('./data/clean_data.csv','w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(row)
    for i in range(len(data)):
        row = [data['/Users/kibaekjeong/Desktop/cloning data/IMG/center_2019_09_06_17_43_40_849.jpg'][i][40:],data['/Users/kibaekjeong/Desktop/cloning data/IMG/left_2019_09_06_17_43_40_849.jpg'][i][40:],data['/Users/kibaekjeong/Desktop/cloning data/IMG/right_2019_09_06_17_43_40_849.jpg'][i][40:],data['0'][i],data['0.1'][i],data['0.2'][i],data['7.420227E-06'][i]]
        writer.writerow(row)
csvFile.close()

In [155]:
clean_data = pd.read_csv(os.path.join(data_dir,'clean_data.csv'))

In [156]:
clean_data.head()

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2019_09_06_17_43_40_926.jpg,IMG/left_2019_09_06_17_43_40_926.jpg,IMG/right_2019_09_06_17_43_40_926.jpg,-0.013971,0.034260,0.0,0.001163
1,IMG/center_2019_09_06_17_43_40_994.jpg,IMG/left_2019_09_06_17_43_40_994.jpg,IMG/right_2019_09_06_17_43_40_994.jpg,-0.028493,0.283845,0.0,0.139888
2,IMG/center_2019_09_06_17_43_41_064.jpg,IMG/left_2019_09_06_17_43_41_064.jpg,IMG/right_2019_09_06_17_43_41_064.jpg,-0.033318,0.481680,0.0,0.465651
3,IMG/center_2019_09_06_17_43_41_142.jpg,IMG/left_2019_09_06_17_43_41_142.jpg,IMG/right_2019_09_06_17_43_41_142.jpg,-0.034926,0.692577,0.0,1.022985
4,IMG/center_2019_09_06_17_43_41_209.jpg,IMG/left_2019_09_06_17_43_41_209.jpg,IMG/right_2019_09_06_17_43_41_209.jpg,-0.038695,0.881505,0.0,1.648629


In [157]:
len(clean_data)

48197

### Model for Behavioral cloning

In [164]:
class generator():
    def load(data_dir,img_path):
        return mpimg.imread(data_dir+img_path)
    def rgb2yuv(img):
        # Transform RGB to YUV color space.
        return cv2.cvtColor(img,cv2.COLOR_RGB2YUV)
    def random_choose(data_dir,center,left,right,steering):
        rand = np.random.choice(3)
        if rand == 0:
            return generator.load(data_dir,center),steering
        elif rand == 1:
            return generator.load(data_dir,left),(steering + 0.2) #correction for left image
        elif rand == 2:
            return generator.load(data_dir,right),(steering - 0.2) #correction for right image
    
    def flip(img,steering):
        rand = np.random.rand()
        #flip the image with 50% chance
        if rand > 0.5:
            img = cv2.flip(img,1)
            steering *= -1
        return img, steering
    
    def brightness(img):
        rand = np.random.rand()
        hsv = cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
        if rand > 0.2:
            rand_bright = 0.5 + np.random.uniform()
            hsv[:,:,2] = hsv[:,:,2]* rand_bright
            hsv[:,:,2][hsv[:,:,2]>255]=255
        return cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)
    
    def generate(data_dir,center,left,right,steering):
        img, steering_angle= generator.random_choose(data_dir,center,left,right,steering)
        img, steering_angle = generator.flip(img,steering_angle)
        img = generator.brightness(img)
        return img, steering_angle
    
    def batch_generator(data_dir,img_paths,steering,batch_size,training=True):
        imgs = np.empty([batch_size,160,320,3])
        steering_angles = np.empty(batch_size)
        while True:
            for i in range(batch_size):
                index = np.random.randint(low=0,high=img_paths.shape[0])
                center,left,right = img_paths[index]
                steering_angle = steering[index]
                if training == True:
                    img, steering_angle = generator.generate(data_dir,center,left,right,steering_angle)
                else:
                    img = generator.load(data_dir,center)
                imgs[i]=generator.rgb2yuv(img)
                steering_angles[i]=steering_angle
            yield (imgs,steering_angles)


In [165]:
def data(param):
    clean_data = pd.read_csv(os.path.join(param.data_dir,'clean_data.csv'))
    X = clean_data[['center','left','right']].values
    y = clean_data['steering'].values
    X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=param.test_size,random_state=param.random_state)
    return X_train,X_valid,y_train,y_valid
    

In [166]:
def model(param):
    model = Sequential()
    model.add(Lambda(lambda x:(x/255.0)-0.5,input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((70,25),(0,0))))
    model.add(Conv2D(24,(5,5),activation='elu',strides=(2,2)))
    model.add(Conv2D(36,(5,5),activation='elu',strides=(2,2)))
    model.add(Dropout(rate = param.drop_rate))
    model.add(Conv2D(48,(5,5),activation='elu',strides=(2,2)))
    model.add(Conv2D(64,(3,3),activation='elu'))
    model.add(Dropout(rate = param.drop_rate))
    model.add(Conv2D(64,(3,3),activation='elu'))
    model.add(Dropout(rate = param.drop_rate))
    model.add(Flatten())
    model.add(Dense(100,activation='elu'))
    model.add(Dense(50,activation='elu'))
    model.add(Dense(10,activation='elu'))
    model.add(Dense(1))
    model.summary()
    return model



In [170]:
def model_train(model,param,X_train,X_valid,y_train,y_valid):
    checkpoint = ModelCheckpoint('model-{epoch:02d}.h5',
                                 monitor = 'val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='auto')
    
    model.compile(loss='mean_squared_error',optimizer=Adam(lr=param.lr))
    
    model.fit_generator(generator.batch_generator(param.data_dir,X_train,y_train,param.batch_size,True),
                        samples_per_epoch=param.samples_per_epoch,
                        epochs=param.epoch,
                        validation_data = generator.batch_generator(param.data_dir,X_valid,y_valid,param.batch_size,False),
                        verbose =1,
                        validation_freq=1,
                        validation_steps = param.samples_per_epoch,
                        callbacks=[checkpoint])
    

In [171]:
def main():
    #Set parameter values
    param = easydict.EasyDict({
        "data_dir":'./data/',
        "drop_rate":0.35,
        "epoch":150,
        "batch_size":32,
        "lr":0.0001,
        "test_size":0.2,
        "random_state":42,
        "samples_per_epoch":5000
    })
    
    X_train,X_valid,y_train,y_valid = data(param)
    driving_model = model(param)
    model_train(driving_model,param,X_train,X_valid,y_train,y_valid)
    

In [172]:
main()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_29 (Lambda)           (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_29 (Cropping2D)   (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_141 (Conv2D)          (None, 31, 158, 24)       1824      
_________________________________________________________________
conv2d_142 (Conv2D)          (None, 14, 77, 36)        21636     
_________________________________________________________________
dropout_42 (Dropout)         (None, 14, 77, 36)        0         
_________________________________________________________________
conv2d_143 (Conv2D)          (None, 5, 37, 48)         43248     
_________________________________________________________________
conv2d_144 (Conv2D)          (None, 3, 35, 64)       

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=200, validation_data=<generator..., verbose=1, validation_freq=1, validation_steps=5000, callbacks=[<keras.ca..., steps_per_epoch=5000)`


Epoch 1/200
5000/5000 [==============================] - 1464s 293ms/step - loss: 0.0268 - val_loss: 0.0176

Epoch 00001: saving model to model-01.h5
Epoch 2/200
5000/5000 [==============================] - 1587s 317ms/step - loss: 0.0205 - val_loss: 0.0182

Epoch 00002: saving model to model-02.h5
Epoch 3/200
5000/5000 [==============================] - 1564s 313ms/step - loss: 0.0190 - val_loss: 0.0159

Epoch 00003: saving model to model-03.h5
Epoch 4/200
5000/5000 [==============================] - 1526s 305ms/step - loss: 0.0177 - val_loss: 0.0160

Epoch 00004: saving model to model-04.h5
Epoch 5/200
5000/5000 [==============================] - 1579s 316ms/step - loss: 0.0172 - val_loss: 0.0151

Epoch 00005: saving model to model-05.h5
Epoch 6/200
5000/5000 [==============================] - 1624s 325ms/step - loss: 0.0167 - val_loss: 0.0154

Epoch 00006: saving model to model-06.h5
Epoch 7/200
5000/5000 [==============================] - 1553s 311ms/step - loss: 0.0164 - val_loss:

5000/5000 [==============================] - 1207s 241ms/step - loss: 0.0113 - val_loss: 0.0124

Epoch 00109: saving model to model-109.h5
Epoch 110/200
5000/5000 [==============================] - 1209s 242ms/step - loss: 0.0116 - val_loss: 0.0123

Epoch 00110: saving model to model-110.h5
Epoch 111/200
5000/5000 [==============================] - 1214s 243ms/step - loss: 0.0117 - val_loss: 0.0124

Epoch 00111: saving model to model-111.h5
Epoch 112/200
5000/5000 [==============================] - 1208s 242ms/step - loss: 0.0112 - val_loss: 0.0125

Epoch 00112: saving model to model-112.h5
Epoch 113/200
5000/5000 [==============================] - 1208s 242ms/step - loss: 0.0111 - val_loss: 0.0126

Epoch 00113: saving model to model-113.h5
Epoch 114/200
5000/5000 [==============================] - 1275s 255ms/step - loss: 0.0110 - val_loss: 0.0104

Epoch 00114: saving model to model-114.h5
Epoch 115/200
5000/5000 [==============================] - 1278s 256ms/step - loss: 0.0112 - val


Epoch 00162: saving model to model-162.h5
Epoch 163/200
5000/5000 [==============================] - 1096s 219ms/step - loss: 0.0112 - val_loss: 0.0136

Epoch 00163: saving model to model-163.h5
Epoch 164/200
5000/5000 [==============================] - 1097s 219ms/step - loss: 0.0112 - val_loss: 0.0138

Epoch 00164: saving model to model-164.h5
Epoch 165/200
5000/5000 [==============================] - 1095s 219ms/step - loss: 0.0109 - val_loss: 0.0140

Epoch 00165: saving model to model-165.h5
Epoch 166/200
5000/5000 [==============================] - 1094s 219ms/step - loss: 0.0111 - val_loss: 0.0142

Epoch 00166: saving model to model-166.h5
Epoch 167/200
5000/5000 [==============================] - 1094s 219ms/step - loss: 0.0112 - val_loss: 0.0138

Epoch 00167: saving model to model-167.h5
Epoch 168/200
5000/5000 [==============================] - 1094s 219ms/step - loss: 0.0109 - val_loss: 0.0137

Epoch 00168: saving model to model-168.h5
Epoch 169/200
5000/5000 [==============